In [1]:
import pandas as pd
import numpy as np
from os import path
from pymodulon.io import *
from pymodulon.plotting import *

from scipy.stats import ks_2samp

In [2]:
data_dir = path.join('..','data','processed_data')
interim_dir = path.join('..','data','interim')
external_dir = path.join('..','data','external')
figure_dir = path.join('..','figures')

In [3]:
ica_data = load_json_model(path.join(interim_dir,'bd.json.gz'))

Sample grouping

In [4]:
groups={
        "102625:110spc4_aerobic _PSY":"Aerobic",
        "102625:110spc4_micro_PSY":"Microaerobiosis",
#         "102625:110spc4_nodules_soybean_21":"Bacterioid",
#        "102625:110spc4_anoxic_YEM":"Anoxic",
#         "101727:110spc4_genistein":"Genistein",
        
       }

Differential activity analysis

In [5]:
selected_A=pd.DataFrame()

refer_id = ica_data.sample_table[ica_data.sample_table.full_name=="102625:110spc4_aerobic _PSY"].index.to_list()
refer_A = ica_data.A[refer_id]
for i in groups:
    sample_id = ica_data.sample_table[ica_data.sample_table.full_name==i].index.to_list()

    temp_A= pd.DataFrame()
    
    for k in sample_id:
        temp_A[k]=ica_data.A[k]
    
    s = pd.Series(dtype="bool_")
    for j in ica_data.imodulon_names:
        if ks_2samp(refer_A.loc[j].values,temp_A.loc[j].values)[1] < 0.05:
            s[j] = True
        else:
            s[j] = False
    selected_A[i]=s

In [6]:
selected_B=pd.DataFrame()

refer_id = ica_data.sample_table[ica_data.sample_table.full_name=="102625:110spc4_aerobic _PSY"].index.to_list()
refer_B = ica_data.A[refer_id]
for i in groups:
    sample_id = ica_data.sample_table[ica_data.sample_table.full_name==i].index.to_list()
    
    temp_B= pd.DataFrame()
    
    for k in sample_id:
        temp_B[k]=ica_data.A[k]
    
    selected_B[i]=temp_B.mean(axis=1) - refer_B.mean(axis=1) 
    

Activated iModulons

In [7]:
sig_index = selected_A[selected_A.iloc[:,1]].index
up_index = selected_B[selected_B.iloc[:,1] > 0].index
active_im_index = sig_index.intersection(up_index)

In [8]:
active_im_index

Index(['FixK2-1', 'NifA/RpoN', 'Unchar-3', 'Irr', 'Unchar-6', 'Unchar-7',
       'HrcA', 'Signal_transduction', 'Leucine', 'Translation-1',
       'Calvin_cycle', 'Oxidation_stress', 'FixK2-2', 'Unchar-11', 'Null-2',
       'Unchar-13', 'NodD2', 'Cell_mobility', 'T6SS', 'Sulfur-1', 'Null-3'],
      dtype='object')

active_TRN

In [9]:
ls1= []
for i in active_im_index:
    im_gene_table = ica_data.view_imodulon(i)[ica_data.view_imodulon(i).regulator.notnull()]
    for k in im_gene_table[["regulator"]].iterrows():
        for j in k[1].str.split(",")["regulator"]:
            ls1.append([k[0],j])

active_TRN = pd.DataFrame(ls1).drop_duplicates()
active_TRN.columns= ["gene","regulator"]

In [10]:
# active_TRN.to_csv("Microaerobiosis_TRN.csv",index=None)

# active_TRN.to_csv("Bacterioid21_TRN.csv",index=None)
# 
# active_TRN.to_csv("Denitrification_TRN.csv",index=None)
# 
# active_TRN.to_csv("Genistein_TRN.csv",index=None)